# Procesamiento del lenguaje natural

* [Práctico 1](#Práctico-1)
* [Práctico 2](#Práctico-2)
* [Práctico 3](#Práctico-3)

## Práctico 1

### Ejercicio 1: Corpus

#### Elegir corpus de texto en lenguaje natural de más de 5Mb de tamaño.

Se elegió el conjunto de reportes de experiencias [Erowid](http://erowid.org), una colección de más de 20.000 reportes personales que describen experiencias con plantas y drogas psicoactivas.

In [4]:
CORPUS_DIR = '../corpus'

Algunos archivos no están en UTF-8 válido así que los corregimos.

In [ ]:
#import os, io

#CORPUS_SOURCE = '../icowid-generator/corpi/text'

#for filename in os.listdir(CORPUS_SOURCE):
    
#    with io.open(CORPUS_SOURCE + '/' + filename, 'r', encoding='utf8', errors='ignore') as i:
#        with io.open(CORPUS_DIR + '/' + 'utf8_' + filename, 'w', encoding='utf8') as o:
#            o.write(i.read())

#### Cargar el corpus usando un “corpus reader” de NLTK (e.g. PlaintextCorpusReader) o definiendo uno propio.

El “corpus reader” debe proveer un método sents que permita iterar sobre las oraciones tokenizadas del corpus.

In [518]:
from nltk.corpus import PlaintextCorpusReader
#nltk.download('punkt')

corpus = PlaintextCorpusReader(CORPUS_DIR, '.*.txt')

#### Revisar a ojo la tokenización y segmentado en oraciones. Si es muy mala, probar otras formas de tokenización/segmentado.

In [522]:
print(list(corpus.sents(fileids=['utf8_erowid_2.txt']))[:3])

[['At', 'first', 'it', 'was', 'just', 'another', 'rave', 'with', 'my', 'crew', '.'], ['Drop', 'a', 'little', 'E', ',', 'have', 'a', 'good', 'night', '.'], ['What', 'I', 'didn', "'", 't', 'know', 'was', 'this', 'was', 'going', 'to', 'be', 'one', 'rave', 'I', 'probably', 'won', "'", 't', 'ever', 'forget', ',', 'what', 'I', 'can', 'remember', 'that', 'is', '.']]


La tokenización parece funcionar bastante bien. Sin embargo se observan algunos defectos producto de utilizar el tokenizador por defecto basado en `nltk.tokenize.regexp.WordPunctTokenize`, que corresponde a la expresión regular `\w+|\[^\w\s]+`.

* Abreviaturas: `['a', '.', 'm', '.']`
* La hora: `['6', ':', '30']`

Más algunas separaciones que podrían repararse o no dependiendo del criterio elegido. Por ahora las dejamos como están.

* Palabras compuestas: `['elastic', '-', 'like']`
* Omisiones, uso del apóstrofe en general: `['couldn', "'", 't']`, `['bustin', "'"]`

Escribimos una expresión regular con más reglas.

**Importante**

* Para que funcione las reglas tienen que ir de más específicas a más generales.
* Los grupos tienen que ser no capturantes `(?: ...)`.

In [2]:
pattern = r'''(?x)    # verbose regexps
      \d+[.:]\d+      # horas y números con decimales
    | \w+\.(?:\w+\.)+ # abreviaturas
    | \w+             # palabras alfanuméricas
    | [^\w\s]+        # signos de puntuación
'''

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(pattern)

tokenizer.tokenize('Son las 6:30 a.m..')

['Son', 'las', '6:30', 'a.m.', '.']

Deja mucho que desear el filtro de archivos (argumento `fileids`) de `PlaintextCorpusReader` así que se los especificamos a mano.

In [5]:
import os, glob
from nltk.corpus import PlaintextCorpusReader

archivos = [os.path.basename(file) for file in glob.glob(CORPUS_DIR + '/utf8_*.txt')]

corpus = PlaintextCorpusReader(CORPUS_DIR, archivos, word_tokenizer=tokenizer)

#### Modificar el script train.py para utilizar el nuevo corpus.

In [99]:
!python train.py -n 1 -o modelo

### Ejercicio 2: Modelo de n-gramas

#### Notas

* `<s>` no se cuenta como unigrama.
* La probabilidad condicional cuando hay división por cero se corrige a cero.
* Los primeros tokens de una oración son `<s> * (n-1)`.

In [8]:
import imp
import ngram

In [213]:
imp.reload(ngram)
ng = ngram.NGram(4, corpus.sents())

### Ejercicio 3: Generación de texto

In [214]:
import ngram_generator
imp.reload(ngram_generator)

ngg = ngram_generator.NGramGenerator(ng)

In [373]:
from subprocess import call

call('rm oraciones.txt', shell=True)

for n in [1,2,3,4]:
    call('python train.py -n {n} -o modelo_n{n}.pkl'.format(n=n), shell=True)
    call('echo "#### Modelo n={n} ####" >> oraciones.txt'.format(n=n), shell=True)
    call('python generate.py -i modelo_n{n}.pkl -n 4 >> oraciones.txt'.format(n=n), shell=True)

#### Modelo n=1 ####

* I
* The ) I truly she the one process. Set time told just it. Of all rolled in a as
* About anywhere cloudy mention we worse having
* I minutes the experiences that least to I it the was this. Dance. Effects verbatim in i wasn days at could is t could

#### Modelo n=2 ####

* I felt that a body, j's.
* Might be okay so much acid hell, and make out to the dealer again.
* A druggy circle', it, and pretty much as I felt an hour or 2 - thc is the time and waited a certain doses of previous night in love for a punishment for granted and I'burned out for so she can't hungry.
* After the right, hoping and 100mg himself.

#### Modelo n=3 ####

* He pointed, and we resolved to keep'.
* Note that this was the worst part of my mates for the past, present and that the feeling of relaxation.
* I have tried and many experiments with datura, aka jimsonweed, belladonna, etc.
* The solvents used in that single experience, which had a deep dreamless sleep.

#### Modelo n=4 ####

* I was having trouble moving around at any kind of trouble or bad experiences I have had with just taking normal Adderall.
* I started writhing on my bed.
* I remember letting go the cloud of smoke.
* I recognized my room, a bed room.

### Ejercicio 4: Suavizado *add-one*

In [342]:
import ngram
imp.reload(ngram)
aong = ngram.AddOneNGram(2, corpus.sents())

### Ejercicio 5: Evaluación de modelos de lenguaje

### Ejercicio 6: Suavizado por interpolación

In [6]:
corpus_sents = [[word.lower() for word in sent] for sent in corpus.sents()]

In [ ]:
import ngram
imp.reload(ngram)
ing = ngram.InterpolatedNGram(3, corpus_sents, gamma=1, addone=True)

In [ ]:
for n in [1,2,3,4]:
    ngram.InterpolatedNGram(n, corpus_sents, addone=True)

```
n | gamma | perplejidad
-----------------------
1 | 0.01  | 542
2 |  100  | 172
3 |  100  | 161
4 | 1000  | 163
```